# 情報科学演習II アルゴリズムとデータ構造II 第1週 4冊目

#### 第1週目次

1. ColabraoryのPythonノートブックに慣れる（1冊目）
2. グラフの最短経路問題を解く（2冊目）
3. グラフ操作ライブラリnetworkxに慣れる（2冊目）
4. リストと集合と辞書（3冊目）
5. 要素の探索にかかる速度を比べる（3冊目）
6. ダイクストラのアルゴリズムを理解する（4冊目）

このノートブック（6章）にコードセルはありません．
読んだりノートに書いたりして理解してください．

## 6. ダイクストラのアルゴリズムを理解する

### 6.1 グラフ
扱うグラフ(graph)は **頂点(vertex)** と **辺(edge)** からなるグラフです（頂点を節点，辺を枝と呼ぶこともあります）．
離散数学IとIIで学んだグラフです．
棒グラフや折れ線グラフではありません．

グラフの例として次のグラフ$G_1$を示します．

![graph](./figs/graph-complex.svg)

このグラフ$G_1$には0から7の8つの頂点があります．

辺は二つの頂点を結んでいる線のことで，16本あります．
数えてみてください．

辺には**長さ**が付いています．
頂点0と4の間の距離は9といった具合です．

用語や記法を説明します．例は上のグラフについての表現です．
- グラフを$G$，頂点の集合を$V$，辺の集合を$E$で表すとき，$G=(V,E)$と書く．
  
  例：$G_1=(V_1,E_1)$，$V_1=\{0, 1, \ldots, 7\}$，$E_1=\{(0, 1), (0, 4), (0, 7), (1, 2), \ldots\}$．

  補足：カッコは2項の組を表す．1項目が頂点の集合，2項目が辺集合とする．
- 二つの頂点$u$と$v$を結ぶ辺を$(u,v)$のように表記する．
  
  例：グラフ$G_1$に辺(0,1)と(0,4)が存在する．
  辺(0,2)は存在しない．
- 辺の向きを考えるグラフを**有向グラフ**，向きを考えないグラフを**無向グラフ**という．
  
  補足：この演習では無向グラフを扱う．
- 辺$(u,v)$は頂点$u$と頂点$v$に**接続している**(incident on)という．
  
  例：グラフ$G_1$の辺(0,4)は頂点0に接続している．
  辺(0,4)は頂点1に接続していない．
- 辺$(u,v)$があるとき，頂点$v$は頂点$u$に**隣接している**(adjacent)という．

  例：グラフ$G_1$の頂点4は0に隣接している．
  頂点4は1に隣接していない．
- $i=1, 2, \ldots, k$に対して辺$(v_{i-1},v_i)$がすべて存在するとき，$k+1$個の頂点の列<$v_0,v_1,v_2, \ldots, v_k$>を**経路**(あるいは道．path)という．
  
  例：グラフ$G_1$の<0,4,5,6>は経路である．
  <0,4,7,6>は辺(7,6)がないので経路ではない．
- 経路<$v_0,v_1,v_2, \ldots, v_k$>について，最初の頂点$v_0$を **始点**と呼び，最後の頂点$v_k$を**終点**と呼ぶ．
  
  例：グラフ$G_1$の経路<0,4,5,6>の始点は0，終点は6である．
  頂点4は，経路<0,4,5,6>の始点ではないが，経路<4,5,6>の始点である．
- 辺$(u,v)$には**長さ**(あるいは距離)がある．この辺の長さを$l_{(u,v)}$で表す．
  
  例：グラフ$G_1$の辺(0,4)の長さは9．$l_{(u,v)}=9$．

  補足：辺に長さを書かないグラフもある．
  そのようなグラフではすべての辺の長さが均一で，どの辺の長さも1とみなす．
- 経路の長さは，経路上の辺の長さの合計とする．
  
  例：グラフ$G_1$の経路<0,4,5,6>の長さは26である．
  $l_{(0,4)}+l_{(4,5)}+l_{(5,6)}=9+4+13=26$．

このグラフは何に見えますか？

- 道路地図：交差点が頂点で道が辺．カーナビや信号機の制御などの交通管制に使えそうです．
- 友人関係：人が頂点で交友関係が辺．LINEやFacebookなどのSNSはこのグラフを管理しています．
- インターネットや電話網：ルータや交換機が頂点でその間の配線が辺．皆さんの通信を支えています．
- 海路や空路：港が頂点で路が辺．

色々な対象をイメージしながら，短い経路を見つける応用を想像してください．

### 6.2 最短経路問題
最短経路問題(shortest path problem)は，グラフ$G=(V,E)$とそれぞれの辺$e\in E$の長さ$l_e$が与えられ，グラフ$G$上の二つの頂点が経路の始点と終点として与えられたとき，この2頂点を結ぶ経路のうち長さが最も短い経路を求める問題である．

最短経路を求ますが，この演習では求めた経路の距離だけ出力します．

二つの頂点$u,v$が与えられて，$u$を始点，$v$を終点とする最短経路を求める問題を，2頂点の単一組に対する最短経路問題(single-pair shortest path problem)といいます．

始点$s$が与えられて，他のすべての頂点への最短経路を求める問題を，単一始点最短経路問題(single-source shortest path problem)と呼びます．
この演習ではこの単一始点最短経路問題を効率的に解くアルゴリズムを学びます．

すべての頂点の組合せに対して最短経路を求める問題もあります．

先ほどのグラフについて，始点$s=0$，終点$t=6$の単一組$(s,t)$に対する最短経路問題を考えましょう．
答えは経路<0,4,5,2,6>であり，この経路の長さは25です．
この経路を次の図では太い辺で示しています．

![](./figs/graph-s0-t6.svg)

始点0から終点6への最短経路の求め方は後で考えることにして，ひとまず長さが25より短い経路がないか探しなさい．
長さが25の別の経路がないかも探しなさい．

長さが25以下の経路がないことがわかりましたか？

以下に5つの経路例と，それぞれの距離を示します．
これらの経路は次の図で赤や青の矢印で指すので，確かめなさい．

- 経路<0,4,6>の距離は29
- 経路<0,1,3,6>の距離は29
- 経路<0,7,2,6>の距離は26
- 経路<0,7,5,6>の距離は27
- 経路<0,4,5,2,6>の距離は25

![](figs/graph-s0-t6-paths.svg)

このほかにも<0,1,3,2,7,4,5,6>や<0,4,5,2,5,2,6>のようにたくさんの経路があります．最短でない経路は気にしないことにします．

#### 6.2.1 単一始点最短経路問題を勘で解く

次のグラフ$G$について，始点$s=0$と定めて，単一始点最短経路問題を解いてください．
求めたいのは始点0からそれぞれの頂点までの最短距離です．
ノートにグラフを写すことから始めましょう．

![](figs/graph-simple.svg)

各自試行錯誤してください．
得るものが多いです．

以下に試行錯誤した例を示します．頂点0から頂点$i$の距離を，$d[i]$に格納し，図では頂点の側に青い文字で示しています．

1. 始点0から遠そうな頂点5を終点として，頂点の数が少ない経路<0,1,5>の距離21を求める．

   ![](figs/i-1.svg)

1. 求めた距離を頂点5の短い距離$d[5]=$21としてメモする．
   ついでに，経路の途中の頂点1についても距離$d[1]＝16$をメモする．

   ![](figs/i-2.svg)

1. 経路<0,4,5>の距離を求めて$d[4]$と$d[5]$にメモする．
   $d[5]$のように複数の経路があるときは短い経路の長さだけ残す．

   ![](figs/i-3.svg)

1. 三角形<0,4,3,0>を構成する経路<0,4>と<0,3,4>の長さを比べると，<0,3,4>がより短い．
   $d[4]$を7に更新する．$d[5]$も8に減らすのを忘れずに．

   ![](figs/i-4.svg)

1. <0,3,4>が短い経路なので<0,3,4,1>と<0,3,4,5,2>の距離を$d[1]$と$d[2]$に入れておく．

   ![](figs/i-5.svg)

1. 始点を0としてすべての頂点への経路の長さが求められた．最短の経路かは分からないが，短いものを選んでいる．

   ![](figs/i-6.svg)

   求めた最短距離を表にまとめると次のように表せる．

| 頂点 | 0 |  1 |  2 |  3 |  4 |  5 |
| ---  | - | :-: | :-: | - | - | - |
| 距離 | 0 | 14 | 14 |  4 |  7 |  8 |

7. 三角形<0,3,4>や<0,1,4>，<1,4,5>は最も長い一辺よりほかの2辺の和が小さいので，和が小さい2辺を通る経路の方が短いだろう．．．


#### 6.2.2 単一始点の最短経路を漏れなく調べて問題を解く

同じグラフ$G=(V,E)$に対して始点$s=0\in V$についての単一始点最短経路問題を解きます．
前節では勘で解きましたが，漏れなく調べたかというとどうでしょうか？
漏れがあるとは最短距離より長い経路を選んで最短だと思い込んだということです．

この節では，漏れなく調べる方法として幅優先探索をしてみます．
ある頂点$u\in V$への短い経路<0, ..., $u$>がわかっているときに，$u$に隣接する頂点$v$を終点とする経路<0, ..., $u, v$>を計算し，この経路が短い経路であれば記録し，さらに探索します．
幅優先で探索するので，短い経路が見つかった場合，この経路の終点$v$をキューに入れておき，のちほどこの頂点$v$に対してさらに$v$に隣接する頂点への経路を探索します．

キューに頂点がなくなったら漏れなく調べたと判断します．

![](figs/graph-simple.svg)

6.2.1項と同じグラフと始点です．
まず以下の解説を読む前に自分のノートに書いて試してください．

図中の青字で書いた$d[0]=0$や∞は始点0からその頂点までの短い経路の距離です．
経路が見つかっていない頂点は最短経路長を∞(無限大)と考えることにします．
調べている頂点$u$とそこから伸びる辺($u,v$)や経路長は赤字で示します．

1. 始点0をキューに追加する．

   ![](figs/j-1.svg)

1. 始点0の隣接頂点1,3,4の距離を計算する．頂点1,3,4にはじめて経路<0,1>, <0,3>, <0,4>が見つかったので，これらの頂点をキューに追加する．

   ![](figs/j-2a.svg)

1. 太線の経路が始点0から短い距離で行ける経路．これらの経路は最短ではないかもしれないので，キューの先頭からから順に，その頂点を通って隣接する頂点までいく経路を調べる．

   ![](figs/j-2b.svg)


4. キューから頂点1を取り出し，頂点1の隣接頂点0,2,4,5について，経路<0,1>を通る経路の距離を計算する．経路<0,1,2>と経路<0,1,5>が短い経路として見つかったので，頂点2と5をキューに追加する．<0,1,0>と<0,1,4>の経路は短い経路ではないので頂点0と4はキューに追加しない．

   ![](figs/j-3.svg)

1. キューから頂点3を取り出し，頂点3の隣接頂点0,4について，経路<0,3>を通る経路の距離を計算する．経路<0,3,4>がこれまでの経路<0,4>より短い経路なので，$d[4]$を更新する．頂点4はすでにキューにあるで，改めて4をキューに追加する必要はない．

   ![](figs/j-4.svg)

1. キューから頂点4を取り出し，頂点4の隣接頂点0,1,3,5について，経路<0,3,4>を通る経路の距離を計算する．経路<0,3,4,1>と<0,3,4,5>がこれまでの経路より短い経路なので，$d[1]$と$d[5]$を更新する．キューにない頂点1をキューに追加する．

   ![](figs/j-5.svg)

1. キューから頂点2を取り出し，頂点2の隣接頂点1,5について，経路<0,1,2>を通る経路の距離を計算する．いずれもこれまでより長い経路なので距離とキューは更新しない．

   ![](figs/j-6.svg)


8. キューから頂点5を取り出し，頂点5の隣接頂点1,2,4について，経路<0,3,4,5>を通る経路の距離を計算する．頂点1と2への経路はこれまでより短い経路なのでd[1]とd[2]を更新する．キューにない頂点2をキューに追加する．

   ![](figs/j-7.svg)

1. キューから頂点1を取り出し，頂点1の隣接頂点0,2,4,5について，経路<0,3,4,5,1>を通る経路の距離を計算する．4つの経路はいずれもこれまでより長い経路しか見つからなかった．キューに追加する頂点は見つからなかった．

   ![](figs/j-8.svg)

1. キューから頂点2を取り出し，頂点2の隣接頂点1,5について，経路<0,3,4,5,2>を通る経路の距離を計算する．どちらの経路も長かった．キューに追加する頂点は見つからなかった．

   ![](figs/j-9.svg)

1. キューに頂点がなくなったので，すべての頂点について最短距離を求めた．最短経路として長さ7の経路<0,3,4>や長さ13の経路<0,3,4,5,1>などが見つかりこれらは最短経路である．

   ![](figs/j-10.svg)

   求めた最短距離を表にまとめると次のように表せる．

| 頂点 | 0 |  1 |  2 |  3 |  4 |  5 |
| ---  | :-: | :-: | :-: | :-: | :-: | :-: |
| 距離 | 0 | 13 | 14 |  4 |  7 |  8 |


### 6.3 ダイクストラのアルゴリズム

ダイクストラのアルゴリズムは単一始点の最短経路問題を効率よく解く解法です．
このアルゴリズムを次回Pythonのコードにしますので，よく理解しなさい．

#### 6.3.1 アルゴリズム

ダイクストラのアルゴリズムは，最短距離が確定した頂点を一つずつ増やしていき，すべての頂点の距離が確定したら終了するアルゴリズムです．

最短距離が確定した頂点を覚えておくために，グラフ$G=(V,E)$の頂点集合$V$を二つの部分集合に分割します．
一つ目の頂点集合$S$は最短経路の長さが確定した頂点の集合です．
もう一つの頂点集合は$S$以外の集合$Q=V-S$です．

最短距離が確定する頂点は一つずつですから，$S$の要素数$|S|$が一つずつ増えていきます．
つまり，whileループの繰り返しで一つずつ頂点を$S$に追加します．
すべての頂点が$S$に含まれるようになったら終了します．
ただし，始点からの経路が存在しない頂点があるグラフ（非連結グラフ）では，その頂点が$S$に入ることはありません．

whileループの一回の繰り返しで，最短距離が未確定の頂点集合$Q$の中から距離が最も短い頂点を一つを選びます．
選んだ頂点の最短距離を確定させます．
もし距離が最も短い頂点が複数ある場合は，それらのうちどれを選んでも構いません．

このように，何らかの基準に基づいて，各繰り返しで近視眼的に最適な判断を繰り返して解を作り上げていくアルゴリズムは，**貪欲なアルゴリズム**（グリーディーアルゴリズム，greedy algorithm）と呼ばれます．
6.2.2項の方法では一端調べた頂点2を後でもう一度調べることがありました．
ダイクストラのアルゴリズムは最短距離が確定した頂点を選ぶので，このように2度調べるといった後戻り（あるいはやり直し）がありません．

また，ここで示すダイクストラのアルゴリズムにはいくつか制約があります．
辺の長さに負の値がある場合は最短経路を見つけられないことがあります．
長さが負の辺があるグラフの場合は，もっと時間がかかる他のアルゴリズムを使います．

$Q$を格納するデータ構造として**min優先度付きキュー**（最小を優先するキュー, min-priority queue）を使うと高速に動作します．
min優先度付きキューについては今週の5章で扱います．
優先度付きキューは**演習Iのアルゴリズムとデータ構造I**で学んだデータ構造です．

このアルゴリズムを6.3.2項で疑似コードにします．

また，始点からの経路が存在しない頂点が存在するグラフを非連結グラフと言い，非連結グラフの場合は疑似コードの修正が少し必要です．


#### 6.3.2 アルゴリズムの疑似コード

グラフ$G=(V,E)$と各辺$e\in E$の長さ$l_e$, $e\in E$，始点$s$が与えられる．
$s$から頂点$v$までの距離を配列$d[v]$に保持する．
このアルゴリズムが終了したとき$d[v]$には始点$s$から終点$v$への最短距離が格納されている．
$pi[v]$には終点$v$への最短経路上で$v$の直前の頂点が格納されている．

1. 距離$d[v]$を初期化する．
   すべての頂点$v$の距離$d[v]$を無限大にする．
1. すべての頂点を集合$Q$に加える．
1. 始点$s$の距離$d[s]$を0にする．
1. 頂点集合$Q$に含まれる頂点がなくなるまで以下を繰り返す．
   1. 頂点集合$Q$から$d[u]$が最小の頂点$u$を一つ選び，$u$をQから削除する．
   1. 頂点$u$を$S$に加える．
   1. 頂点$u$に隣接する頂点$v$それぞれについて以下の処理をする．
      1. $d[v]$と$d[u]+l_{(u,v)}$のうち，短い方を新たな$d[v]$とする．

次回の後半の実装では，$Q$のデータ構造としてmin優先度付きキューを使うことで効率化する．
上の3の段階で始点$s$がキューの先頭に来るようにキュー内で$s$を移動する．
上の4-1の段階で，頂点集合$Q$から$d[u]$が最小の頂点$u$を選ぶ際，キューの先頭要素が最小値である．
4-3-1の段階で，$d[u]+l_{(u,v)}$の方が短い場合は$Q$の中での順位も変更する．

次回の前半の実装では，$Q$のデータ構造として，min優先度付きキューではなくリストを用いる．
リストの方が実装が簡単になるものの，リストの探索はリストサイズが大きくなると時間がかかるため，実行速度は遅い．

これを疑似コードで書くと次の`dijkstra`関数になる．
次回前半と後半のどちらの実装でもこの関数はほぼ同一である．
```
d = {} # d[v]は頂点sから頂点vまでの距離
Q = [] # 距離が確定していない頂点のリスト
pi = {} # pi[v]は短い経路上で，vの直前にある頂点

# G: グラフ, L: Gの各辺の長さの辞書, s: 始点とする頂点
dijkstra(G, L, s):
  initialize_single_source(G) # すべての頂点uについて距離d[u]を初期化する
  S = set() # Sは空集合
  
  # キューQの初期化
  for 頂点u in V: # Vに含まれるそれぞれの頂点uに対して
    insert(u) # Qに頂点uを加える

  # 始点sの距離を0にする
  d[s] = 0

  # 最短距離を求める
  while Q が空集合でない: # Qが空集合になるまで
    頂点u = extract_min()   #   Qから最小値の頂点uを取り出す
    S に u を加える       #   Sに頂点uを加える
    for 頂点v in uの隣接頂点: #   uに隣接するそれぞれの頂点に対して
      relax(u, v, L) #     頂点vへの二つの経路のうち短いものを残しキューの並びも調整する
```

`dijkstra`関数内で呼び出す4つの関数は，4章のリストを使った実装では以下のようにする．
```
# すべての頂点uについて距離d[u]を初期化する．距離は無限大にする．
initialize_single_source(G):
  for 頂点v in V: # VはグラフGの頂点の集合
    d[v] = 無限大
    pi[v] = -1  # -1を該当する頂点がないという意味で使うことにする

# Qに頂点uを加える
insert(u):
  キューQに頂点uを加える．

# Qから最小値の頂点uを取り出す
extract_min():
  Qから距離d[v]が最も短い頂点vを探して返す．

# 頂点vへの二つの経路のうち短いものを残す
relax(u, v, L):
  頂点uの距離d[u]に辺(u,v)の長さluvを加えた距離をdv2とする．
  もし，dv2がこれまでの距離d[v]より短ければ，dv2をd[v]に代入し，pi[v]にuを代入する．
```

#### 6.3.3 例

これまでと同じグラフ$G$と始点0について，ダイクストラのアルゴリズムで最短経路問題を解いてみます．

1. 初期化．すべての頂点の距離を∞にする．
   最短距離が求まった頂点の集合$S$は空集合にする．
   キュー$Q$にすべての頂点を追加する．
   始点0の距離$d[0]$を0にする．
   
   ![](figs/k-1.svg)

1. キュー$Q$から距離が最短の頂点0を取り出す．
   集合$S$に頂点0を加える．
   頂点0に隣接する頂点1,4,3の距離をそれぞれ求める．
   頂点1,4,3の距離を更新する．
   
   ![](figs/k-2.svg)

1. キュー$Q$から距離が最短の頂点3を取り出す．
   集合$S$に頂点3を加える．
   頂点3に隣接する頂点0,4について，経路<0,3>に辺を一つ加えた経路<0,3,0>と<0,3,4>の距離を求める．
   頂点4の距離を7に更新する．
   頂点0の距離は短くならないので更新しない．
   
   ![](figs/k-3.svg)

1. キュー$Q$から距離が最短の頂点4を取り出す．
   集合$S$に頂点4を加える．
   頂点4に隣接する頂点は0,1,5,3の4つ．
   経路<0,3,4>に頂点0,1,5,3の一つを加えた経路の距離を求める．
   頂点1,5の距離を更新する．
   他の頂点の距離は短くならないので更新しない．
   
   ![](figs/k-4.svg)

1. キュー$Q$から距離が最短の頂点5を取り出す．
   集合$S$に頂点5を加える．
   頂点5に隣接する頂点は1,2,4の3つ．
   経路<0,3,4,5>に頂点1,2,4の一つを加えた経路の距離を求める．
   頂点1,2の距離を更新する．
   頂点4の距離は短くならないので更新しない．

   ![](figs/k-5.svg)

1. キュー$Q$から距離が最短の頂点1を取り出す．
   集合$S$に頂点1を加える．
   頂点1に隣接する頂点は0,2,5,4の4つ．
   経路<0,3,4,5,1>に隣接頂点の一つを加えた経路の距離を求める．
   どの頂点の距離も変わらないので，距離を更新する頂点はない．
   
   ![](figs/k-6.svg)

1. キュー$Q$から最後の頂点2を取り出す．
   集合$S$に頂点2を加える．
   頂点2に隣接する頂点は1,5の2つ．
   経路<0,3,4,5,2>に頂点1,5の一つを加えた経路の距離を求める．
   どの頂点の距離も変わらないので，距離を更新する頂点はない．
   
   ![](figs/k-7.svg)

1. キュー$Q$に頂点がないので終了する．
   すべての頂点が集合$S$に入っている．
   始点0から頂点iへの最短距離はd[i]．
   
   ![](figs/k-8.svg)
   
   求めた最短距離を表にまとめると次のように表せる．

| 頂点 | 0 |  1 |  2 |  3 |  4 |  5 |
| ---  | - | :-: | :-: | - | - | - |
| 距離 | 0 | 13 | 14 |  4 |  7 |  8 |

#### 6.3.4 アルゴリズムの正しさ
<!-- Kevin Wayneのスライド 04GreedyAlgorithmII.pdf p.10 より -->

ダイクストラのアルゴリズムで求められた最短経路より短い経路が存在しないことを帰納法で証明します．
一つの問題を例を交えてじっくり考えてきたので，これまで証明が苦手な人でもこの証明が腑に落ちると思います．
今理解できなくても，後で読み直すとわかるときがきます．

最短距離が確定した頂点$u\in S$の最短距離は$d[u]$です．
一方，最短距離が確定していない頂点$v\in Q$の$d[v]$は，これまで探索した限りでの最短距離です．
つまり，後者の$d[v]$はアルゴリズムの進行にしたがって今後短くなる可能性があります．

不変条件：それぞれの頂点$u\in S$について，$d[u]$は$s$から$u$への最短経路の長さ

証明：（$S$についての帰納法）

$|S|=1$の場合は$S=\{s\}$かつ$d[s]=0$なので明らかに不変条件が成り立つ．

帰納仮説：$|S|\ge 1$の場合に不変条件が成り立つと仮定し，$S$に一つの頂点を加えた後も不変条件を満たす．

- 頂点$v$を新たに$S$に加える頂点とし，$s$から$v$への最短経路の最後の辺を$(u,v)$とする．
- つまり，$s$から$u$への最短経路に辺$(u,v)$を加えた経路が$s$から$v$への経路でその長さを$d[v]$とする．$d[v] = d[u] + l_{(u,v)}$
- $s$から$v$へのこの経路以外の任意の経路$P$を考え,
  経路$P$の長さ$l(P)$が$d[v]$より短くないことを示せれば帰納仮説が正しいと言える．
- $e=(x,y)$を経路$P$の辺のうち初めて$S$を出る辺とし，
  $P$の一部で$s$から$x$への経路を$P'$とする．
- 経路$P$のうち頂点$s$から頂点$y$に至る部分の長さはすでに$d[v]$以上である．

これらのことから任意の$P$について次の不等式が成り立つ．

$$ l(P) \ge l(P') + l_e \ge d[x] + l_e \ge d[y] \ge d[v] $$

この式の不等号が成り立つ理由は，左の不等号から順に
1. 辺の長さが負ではないから，$l(P) \ge l(P') + l_e$．
   言い換えると，$l(P) = l(P') + l_e + l(残りのyからvへの経路)$，かつ$l(残りのyからvへの経路)\ge 0$だから．
3. 不変条件より$l(P')\ge d[x]$．よって，$l(P') + l_e \ge d[x] + l_e$
4. $d[y]$は$s$から$y$へのこれまで探索した限りでの最短経路の長さだから，$d[x] + l_e \ge d[y]$
5. ダイクストラのアルゴリズムが新たに$S$に加える頂点として$y$ではなく$v$を選んだから，$d[y] \ge d[v]$

![](figs/dijkstra-proof.svg)

記号の説明：$l(P)$は経路Pの長さ．
$l_e$は辺$e$の長さ．
$d[x]$は，$x\in S$ならば始点$s$から頂点$x$までの最短距離．
$x\in Q$ならば，$d[x]$は始点$s$から頂点$x$までのこれまで探索した限りでの最短距離．

上の不等式より，頂点$u$への最短経路に辺$(u, v)$を加えた経路の長さが頂点$v$への経路の最短$d[v]$であり，この経路より短い経路$P$は存在しない．
つまり，不変条件を満たす集合$S$に$v$を加えた集合$S + \{v\}$も不変条件を満たす．
帰納仮説は正しい．

（証明終わり）

#### 6.3.5 アルゴリズムの効率的な実装
<!-- Kevin Wayneのスライド 04GreedyAlgorithmII.pdf p.11 より -->

一つ目の最適化は，まだ距離がもとまっていない頂点$v\notin S$について
次の$d(v)$の定義に沿って求める代わりに効率的な方法で求める．
 
$$d(v) = \min_{(u,v): u\in S} d[u] + l_e$$

- アルゴリズムの途中で$|S|$は増加する一方だから，それぞれの頂点$v \notin S$について$d(v)$は増加しない．
- 具体的には，頂点$u$が$S$に追加されるとき，辺$(u,v)$があるならば，次の式で$\pi[v]$を更新すれば十分である．
  $$d[v] \leftarrow \min(d[v], d[u] + l_e)$$
  ここで，それぞれの頂点$u\in S$について，$d[u] = s$から$u$への最短距離であることを思い出そう．

最適化の二つ目は，$S$に含まれない頂点$v$の$d[v]$が最小の頂点を選ぶためにmin優先度キューを使う．
二つ目の最適化は次回の前半では実装せず，後半で実装する．

今週は以上です．